In [2]:
import nltk
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise  import cosine_similarity

In [3]:
data = pd.read_csv('BankFAQs.csv')

In [4]:
data

Question  \
0     What security do I need to provide to obtain a...   
1     What are the benefits of having a Current/Savi...   
2     How long will it take for me to get a Business...   
3        Can I repay the loan earlier for Business Loan   
4     How can I get my address changed in my loan ac...   
...                                                 ...   
1453                                 How are you today?   
1454       I wonder what do you think about developers?   
1455               Can you send commands to other bots?   
1456             Will the robots hurts people sometime?   
1457                                       How are you?   

                                                 Answer     Class  
0     To obtain the loan, there is absolutely no nee...     loans  
1     You get preferred Processing Fee, priority pro...     loans  
2     Once you submit your application form and supp...     loans  
3     Yes, you can choose to repay your loan anytime...     loans  
4     In order to change your address in our records...     loans  
...                                                 ...       ...  
1453                                I'm super! And you?  chitchat  
1454  Coding is like magic, building something from ...  chitchat  
1455                  Only through an API, not directly  chitchat  
1456  Robots are already hurting people, that is so ...  chitchat  
1457                                     Fine, and you?  chitchat  

[1458 rows x 3 columns]

In [40]:
stop_words = stopwords.words('english')
stop_words.extend(string.punctuation)
lemmtizer = WordNetLemmatizer()

def clean_text(text):
    text = [lemmtizer.lemmatize(token) for token in nltk.word_tokenize(text.lower()) if token not in string.punctuation]
    return ' '.join(text)

In [41]:
# stop_words

In [42]:
data['Question_Cleaned'] = data['Question'].apply(clean_text)

In [43]:
data.head()

Question  \
0  What security do I need to provide to obtain a...   
1  What are the benefits of having a Current/Savi...   
2  How long will it take for me to get a Business...   
3     Can I repay the loan earlier for Business Loan   
4  How can I get my address changed in my loan ac...   

                                              Answer  Class  \
0  To obtain the loan, there is absolutely no nee...  loans   
1  You get preferred Processing Fee, priority pro...  loans   
2  Once you submit your application form and supp...  loans   
3  Yes, you can choose to repay your loan anytime...  loans   
4  In order to change your address in our records...  loans   

                                    Question_Cleaned  
0  what security do i need to provide to obtain a...  
1  what are the benefit of having a current/savin...  
2  how long will it take for me to get a business...  
3     can i repay the loan earlier for business loan  
4  how can i get my address changed in my loan ac...

In [44]:
vectorizer = TfidfVectorizer()
vectorizer.fit(data.Question_Cleaned)

TfidfVectorizer()

In [45]:
Question_Cleaned_vector = vectorizer.transform(data.Question_Cleaned)

In [46]:
Question_Cleaned_vector

<1458x1206 sparse matrix of type '<class 'numpy.float64'>'
	with 13812 stored elements in Compressed Sparse Row format>

In [47]:
user_meassage = 'can you please tell me a joke regarding finance!'

In [48]:
user_meassage_vectorizd = vectorizer.transform([clean_text(user_meassage)])

In [49]:
similarities = cosine_similarity(user_meassage_vectorizd, Question_Cleaned_vector)

In [50]:
similarities

array([[0.        , 0.        , 0.13078874, ..., 0.10855178, 0.        ,
        0.22011465]])

In [51]:
idx = similarities.argmax() # index wiith maximum value

In [52]:
idx

1444

In [53]:
similarities[0][idx]

0.7357396554019175

In [54]:
data.iloc[idx]

Question                                               tell me a joke
Answer              A robot walks into a bar, orders a drink, and ...
Class                                                        chitchat
Question_Cleaned                                       tell me a joke
Name: 1444, dtype: object

In [55]:
bot_response = data.iloc[idx,1]

In [56]:
bot_response

"A robot walks into a bar, orders a drink, and lays down some cash.\n\nBartender says, 'Hey, we don't serve robots.'\n\nAnd the robot says, 'Oh, but someday you will.'\n\n😂😂😂"

In [57]:
def get_bot_response(user_message):
    user_meassage_vectorizd = vectorizer.transform([clean_text(user_message)])
    similarities = cosine_similarity(user_meassage_vectorizd, Question_Cleaned_vector)
    idx = similarities.argmax() 
    sim = similarities[0][idx]
    bot_response = data.iloc[idx,1]
    if sim<0.5:
        return 'I am sorry, I am not abble to understand'
    else:
        return bot_response

In [59]:
get_bot_response('how are you?')

'Fine, and you?'

In [ ]:
def run_bot():
    flag= True
    print('Welcome...')
    while flag:
        user_meassage = input('Human: ')
        if user_meassage.lower()== 'bye':
            print('Bot: Bye...')
            break
        else:
            bot_response = get_bot_response(user_meassage)
            print('Bot: ', bot_response)
            print('*'*30)

In [61]:
run_bot()

Welcome...
Human: hello how are you
Bot:  Greetings!
********
Human: how to open savings account?
Bot:  In order to open a new Savings Account, simply walk into the nearest HDFC Bank and speak to a customer service executive. Remember to carry the following documents (original for verification and self-attested copies for submission): Identity Proof Address Proof Latest passport size photographs Click here to see the List of valid identity/addres proof.
********
Human: what is cibil?
Bot:  CIBIL stands for Credit Information Bureau India Limited. CIBIL's aim is to fulfil the need of credit granting institutions for comprehensive credit information by collecting, collating and disseminating credit information pertaining to both commercial and consumer borrowers, to a closed user group of members. Banks, Financial Institutions, Non-Banking Financial Companies, Housing Finance Companies and Credit Card Companies use CIBIL's services. Data sharing is based on the Principle of Reciprocity, 

In [65]:
run_bot()

Welcome...
Human: hello
Bot:  Greetings!
********
Human: bye
Bot: Bye...
